<a href="https://colab.research.google.com/github/yukinaga/twitter_bot/blob/master/section_5/01_input_padding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 入力のパディング
入力となる文章の長さは様々なのですが、バッチ内ではデータの時系列の長さを揃える必要があります。  
今回は、短い文章に対して「パディング」を行い、バッチ内の全ての文章の長さを揃えます。

## テンソルのパディング
`nn.utils.rnn.pad_sequence`でパディングを行うことができます。

サイズが異なる3つのテンソルを用意します。

In [1]:
import torch
import torch.nn as nn

a = torch.ones(3, 5)
print(a)
b = torch.ones(2, 5) * 2
print(b)
c = torch.ones(1, 5) * 3
print(c)

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])
tensor([[2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.]])
tensor([[3., 3., 3., 3., 3.]])


`nn.utils.rnn.pad_sequence`によりパディングを行います。

In [2]:
padded = nn.utils.rnn.pad_sequence([a, b, c], batch_first=True)
print(padded)

tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2.],
         [0., 0., 0., 0., 0.]],

        [[3., 3., 3., 3., 3.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]])


## Tensorのパッキング
このままだと0の入力が多数になってしまうので、RNNで適切に扱うためには「パッキング」を行う必要があります。  
`rnn.pack_padded_sequence`により、0を除いたPackedSequence型のデータを作成できます。  
PackedSequence型のデータは、RNNに入力することが可能です。

In [3]:
packed = nn.utils.rnn.pack_padded_sequence(padded, [3, 2, 1], batch_first=True, enforce_sorted=False)
print(packed)

PackedSequence(data=tensor([[1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3.],
        [1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2.],
        [1., 1., 1., 1., 1.]]), batch_sizes=tensor([3, 2, 1]), sorted_indices=tensor([0, 1, 2]), unsorted_indices=tensor([0, 1, 2]))


## RNNへ入力
パッキングされたTensorをRNNへ入力します。  
出力は`PackedSequence`型のデータに、隠れ層の状態hは通常のTensorになります。

In [4]:
rnn = nn.RNN(
            input_size=5,  # 入力サイズ
            hidden_size=2,  # ニューロン数
            batch_first=True,  # 入力を (バッチサイズ, 時系列の数, 入力の数) にする
        )

y, h  = rnn(packed)
print(y)
print(h)

PackedSequence(data=tensor([[ 0.2670, -0.4402],
        [ 0.4090, -0.6955],
        [ 0.5337, -0.8467],
        [ 0.1869, -0.4419],
        [ 0.2918, -0.6999],
        [ 0.1843, -0.4685]], grad_fn=<CatBackward>), batch_sizes=tensor([3, 2, 1]), sorted_indices=tensor([0, 1, 2]), unsorted_indices=tensor([0, 1, 2]))
tensor([[[ 0.1843, -0.4685],
         [ 0.2918, -0.6999],
         [ 0.5337, -0.8467]]], grad_fn=<IndexSelectBackward>)


## PackedSequence型をTensorに戻す
`pad_packed_sequence`により、RNNの出力をパディングされたTensorに戻します。

In [5]:
y_unpacked = nn.utils.rnn.pad_packed_sequence(y, batch_first=True)
print(y_unpacked)

(tensor([[[ 0.2670, -0.4402],
         [ 0.1869, -0.4419],
         [ 0.1843, -0.4685]],

        [[ 0.4090, -0.6955],
         [ 0.2918, -0.6999],
         [ 0.0000,  0.0000]],

        [[ 0.5337, -0.8467],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]]], grad_fn=<IndexSelectBackward>), tensor([3, 2, 1]))
